<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_9_tuning_bigtweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [12]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    916      0 --:--:-- --:--:-- --:--:--   916
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [13]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

fatal: destination path 'demo-nlp-classification-fastai' already exists and is not an empty directory.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [55]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,3,cracking sensations in my joints
1,5,fast and slow heartbeat
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [56]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-9.test.csv",header=None)
test.head()

,0,1
0,2,painful heels
1,4,Discoloration of skin [blotchy] on the neck
2,5,my heart rate jumped to 106
3,7,goiter
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_bigtweet_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [61]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [66]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.136804,4.271298,0.268814,03:52


In [67]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.091075,3.724304,0.328858,03:51


In [68]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.764627,3.685724,0.332015,03:51


In [69]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.725062,3.651440,0.337341,03:52


In [0]:
learn.save_encoder('bigtweet_fold9_first')

In [71]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold9_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.703911,4.229085,0.335580,01:07


In [72]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.889650,3.871862,0.377358,01:19


In [73]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.616994,3.702204,0.392183,03:32


In [74]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.403857,3.571073,0.412399,03:11
1,3.208029,3.509677,0.431267,03:10


In [75]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.121633,3.425356,0.431267,03:23
1,3.005180,3.363226,0.433962,03:19


In [76]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.989187,3.343719,0.444744,03:20
1,2.900540,3.266572,0.446092,03:35


In [77]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.854249,3.276618,0.447439,03:17
1,2.769665,3.265403,0.415094,03:30


In [78]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.781802,3.262830,0.448787,03:24
1,2.695363,3.175885,0.456873,03:24


In [79]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.627458,3.222269,0.446092,03:21
1,2.666680,3.123351,0.458221,03:28
2,2.635597,3.084746,0.456873,03:33
3,2.542955,3.082570,0.459569,03:26


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.543984,3.143522,0.469003,03:16
1,2.521032,3.092916,0.470350,03:19
2,2.427164,3.090011,0.475741,03:31
3,2.441231,3.109299,0.466307,03:29


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.409689,3.209099,0.452830,03:17
1,2.458503,3.112708,0.469003,03:17
2,2.397495,3.100418,0.473046,03:16
3,2.358595,3.103909,0.467655,03:27


In [82]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.342927,3.189257,0.442049,03:14
1,2.400698,3.163235,0.455526,03:09
2,2.365618,3.088311,0.466307,03:20
3,2.319010,3.188386,0.463612,03:32


In [83]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.254107,3.186160,0.428571,03:18
1,2.407562,3.274923,0.463612,03:34
2,2.406054,3.210650,0.466307,03:21
3,2.232299,3.233726,0.471698,03:22


In [84]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.334792,3.249027,0.470350,03:27
1,2.373163,3.286143,0.454178,03:32
2,2.280375,3.202174,0.470350,03:29
3,2.226499,3.181991,0.467655,03:19


In [85]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.250252,3.239951,0.464960,03:28
1,2.406435,3.301856,0.455526,03:14
2,2.411098,3.378572,0.464960,03:27
3,2.200558,3.253981,0.467655,03:31


In [0]:
# save the best model

learn.save_encoder('ask-9-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [88]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[228, 841, 5, 1026, 1026, 1026, 1026, 1026, 1026, 203, 16, 16, 20, 20, 896, 805, 536, 1035, 27, 749, 414, 507, 414, 289, 775, 864, 864, 1035, 117, 771, 289, 289, 210, 415, 227, 741, 1026, 49, 49, 90, 265, 245, 117, 117, 428, 896, 1035, 896, 426, 50, 374, 502, 289, 289, 893, 741, 896, 896, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 289, 172, 896, 896, 896, 896, 864, 896, 289, 80, 1035, 1035, 416, 207, 1035, 760, 444, 138, 896, 896, 896, 1026, 896, 896, 896, 896, 94, 94, 94, 94, 402, 29, 265, 518, 408, 408, 503, 265, 265, 265, 1035, 774, 864, 864, 864, 983, 864, 741, 604, 864, 278, 983, 117, 415, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 773, 129, 210, 1026, 555, 137, 268, 896, 896, 232, 447, 896, 896, 896, 854, 829, 829, 321, 896, 896, 896, 896, 896, 896, 245, 864, 289, 487, 265, 289, 864, 864, 864, 164, 896, 983, 178, 896, 127, 182, 864, 289, 1035, 298, 298, 298, 183, 1

In [89]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

203
0.23441108545034642
